# Report

## What steps were performed and what steps were skpped? 

The only step I truely skipped was that I had to use read_csv() and to_csv() to load and download the DataFrames created in the script files I created for hyperparamet tuning and model testing. 

Therefore, the code looked like this:
```
# Data for LGBM and XGBoost
train_df_light_xgb = features_train.copy()  
train_df_light_xgb['target'] = target_train  
train_df_light_xgb.to_csv('train_data_light_xgb.csv', index=False)

test_df_light_xgb = features_test.copy()  
test_df_light_xgb['target'] = target_test  
train_df_light_xgb.to_csv('test_data_light_xgb.csv', index=False)

# Data for CatBoost
train_df_cat = features_train.copy()  
train_df_cat['target'] = target_train  
train_df_cat.to_csv('train_data_cat.csv', index=False)

test_df_cat = features_test.copy()  
test_df_cat['target'] = target_test  
test_df_cat.to_csv('test_data_cat.csv', index=False)
```
Instead of like this:
```
# Data for LGBM and XGBoost
train_df_light_xgb = features_train.copy()  
train_df_light_xgb['target'] = target_train  
train_df_light_xgb.to_parquet('train_data_light_xgb.parquet', index=False)

test_df_light_xgb = features_test.copy()  
test_df_light_xgb['target'] = target_test  
train_df_light_xgb.to_parquet('test_data_light_xgb.parquet', index=False)

# Data for CatBoost
train_df_cat = features_train.copy()  
train_df_cat['target'] = target_train  
train_df_cat.to_parquet('train_data_cat.parquet', index=False)

test_df_cat = features_test.copy()  
test_df_cat['target'] = target_test  
test_df_cat.to_parquet('test_data_cat.parquet', index=False)

This was simply due to the fact that pyarrow was not installed on TTT's notebook, and when I tried to run a pip install I got an error message informing me that I did not have permission to make that installation (see below) 

In [1]:
!pip install pyarrow
!pip install shap
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 146.8 MB/s eta 0:00:00a 0:00:01
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/.venv/lib/python3.9/site-packages/pyarrow'
Check the permissions.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.2/997.2 kB 104.6 MB/s eta 0:00:00
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/.venv/lib/python3.9/site-packages/slicer'
Check the permissions.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/3 [slicer]


Otherwise, I performed all of the steps that were written up in the plan. I ran the model with different sets of features and performed a SHAP analysis to discern when and where data leakage was occurring. Then I chose the set of features that lead to the best roc_auc score with no signs of data leakage. Finally, I added the threshold asdjustment to the model that had the best roc_auc score so that the accuracy score would converge on the roc_auc score.

However, there are a couple of things which I did not do that I perhaps should have. Specifically, I did not calculate the recall scores nor optimal decision thresholds for the LightGBM and CatBoost models. In retrospect, this would have been worth doing because even though the XGBoost model had the best roc_auc score during cross validation, it had the worst accuracy score. Moreover, since ultiately the value of the model in this case is the ability to predict which customers are likely to churn, and secondarily which are not, some combination of accuracy and recall really is what's important here. Something I learned as a result of this is that accuracy scores will not always improve in a way that is proportional to a model's roc_auc score with a threshold adjustment. I had assumed this would be the case when I started the project, which is why I was not initially concerned with calculating an optimal threshold and recall score for each model. If I had this project to do over again, I would have calculated the recall score and optimal threshold during the cross validation phase, and selected a model accordinlgy. 


# What difficulties did I encounter? 

The most difficult part of the task was finding a combination of features that optimized performance without leading to data leakage. To solve this problem I ran a SHAP analysis on each model I ran, and, based on that analysis, I tried to come up with a story in each instance of how the given set of features could contribute to data leakage, and how the given set of features could contribute to genunely improved performance.

For example, while it seemed plausible that dayofweek features were not a dead giveaway to begin_date, and could therefore be paired with 'customer_duration' without leading to data leakage, there was also a really plausible story about how it could give away the begin date, and there was not a plausible story about how it could contribute so much predictive power so as to improve the roc_auc score from a .89 to a .99. Such jumps in performance, were, thusly, a major red flag, and ultimately I had to drop all dayofweek features (as well as all other date features) from the data set. 

# What were the key steps to solving the task? 

The key steps to solving the task included engineering the 'customer_duration' and the 'p_i_or_b' feature, although the former contributed way more to the model's decision making according to the SHAP output. 

The 'customer_duration' feature turned out to be especially important because including all of the date features in conjunction with total_charges would have almost certainly led to data leakage due to the heavy correlation between 'total_charges' and 'customer_duration'. However, removing total_charges from that combination of features led to less than optimal performance. Therefore, creating the 'customer_duration' feature allowed us to drop all of the date features without losing performance. Moreover, we can see from the SHAP output that, although there is a heavy correlation between 'customer_durtaion' and 'total_charges', 'total_charges' still made a significant contribution to the model's prediction. 

Another key step to solving the task was subtracting a range of float values from 'customer_duration'. Not only did this help prevent data leakage, subtracting random float values between 12 and 15 allows the model to make predictions based on a churn date that is in the future. This is essential of Interconnect wants to be able to send out promotional offers before a churn takes place. 

See below for the code that was used for this: 

```
# Generate a different random value between 13 and 15 for each row
random_values = np.random.uniform(12, 15, size=len(all_data))

#Check to make sure our random values are what we expect them to be
print(random_values.max())
print(random_values.min())

# Subtract random float values from 'customer_duration
all_data['customer_duration'] = all_data['customer_duration'] - random_values

# What was the final model and what quality score did it have? 

The final model was an XGBoost Classifier with the following hyperparameters:

Best parameters found:  {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'scale_pos_weight': 3, 'subsample': 1.0}

It performed with a roc_auc score of 0.85 and an accuracy score of 0.76 during cross validation. We then calculated an optimal decision threshold to be 0.4946 for positive churn classifications. 

When trained on the full training set with the adjusted decision threshold the model performed with a roc_auc score of 0.90, an accuracy score of 0.79 and a recall score of 0.88. Given that the goal of the task was to predict which customers will churn ahead of time so they can be offered promotional packages, this combination of accuracy and recall efficacy is suitable for the task at hand. 

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> This looks like a good report, nicely done!
<a class="tocSkip"></a>